# About

In [ ]:
print(test)

# Prerequisites

In [ ]:
my_project_id = "idc-pathomics-000"

In [ ]:
from google.colab import auth 
auth.authenticate_user()

# Environment setup

In [ ]:
curr_dir = !pwd
curr_droid = !hostname
curr_pilot = !whoami

print("Current directory :", curr_dir[-1])
print("Hostname          :", curr_droid[-1])
print("Username          :", curr_pilot[-1])

In [ ]:
# Install python-openslide 
%%capture
!sudo apt-get update
!sudo apt-get install --no-install-recommends -y python3-openslide

# Install other requirements or are these already available? 

In [ ]:
# Import all required python libraries for this use case
import os

# Dataset selection and exploration using BiqQuery

In [ ]:
%%bigquery --project=$myProjectID ct_series_counts

WITH
  all_lidc_ct_series AS (
  SELECT
    DISTINCT(SeriesInstanceUID),
    PatientID
  FROM
    `canceridc-data.idc_views.dicom_all`
  WHERE
    Modality = "CT"
    AND collection_id = "lidc_idri")
SELECT
  PatientID,
  COUNT(PatientID) AS ct_series_count
FROM
  all_lidc_ct_series
GROUP BY
  PatientID
ORDER BY
  ct_series_count DESC

In [ ]:
# IDC viewer for pathomics data? 
def get_idc_viewer_url(study_UID):
    return "https://viewer.imaging.datacommons.cancer.gov/viewer/" + study_UID

# Pathomics use case
## Preprocessing

In [ ]:
input_dir = os.environ['IDC_INPUT_DATA_DIR']
slides_dir = os.path.join(input_dir, 'cptac_slides')
tiles_dir = os.path.join(input_dir, 'cptac_tiles')

In [ ]:
from data.tile_generation_cptac import generate_tiles

generate_tiles(slides_dir, tiles_dir, desired_magnification=20.0)

In [ ]:
#from data.tile_sorting import sort_tiles

#metadata_file = os.path.join(input_dir, 'metadata.cart.2017-03-02T00_36_30.276824.json')
#sort_tiles(tiles_dir, metadata_file, input_dir, 'norm_cancer', magnification=5.0)
#sort_tiles(tiles_dir, metadata_file, input_dir, 'luad_lusc', magnification=5.0)
#sort_tiles(tiles_dir, metadata_file, input_dir, 'norm_luad_lusc', magnification=5.0)

## Training
## Evaluation